In [1]:
import os

import pandas as pd
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sensor_data_path = "./sensor_data"

encoder_data = pd.read_csv(os.path.join(sensor_data_path,"encoder.csv"), names=["timestamp", "left_count", "right_count"])
fog_data = pd.read_csv(os.path.join(sensor_data_path,"fog.csv"), names=["timestamp", "delta_roll", "delta_pitch", "delta_yaw"])


# header names for lidar_data
lidar_angle = np.around(np.linspace(-5, 185, 286,),1).astype('str').tolist()
lidar_angle.insert(0, "timestamp")

# lidar_data = pd.read_csv(os.path.join(sensor_data_path,"lidar.csv"),names=lidar_angle)

In [3]:
# first column represents the timestamp of the observation
'''
FOV: 190 (degree)
Start angle: -5 (degree)
End angle: 185 (degree)
Angular resolution: 0.666 (degree) # shoot a laser ray every 0.666 degree
Max range: 80 (meter)

* LiDAR rays with value 0.0 represent infinite range observations.
'''
# print(lidar_data.shape)
# lidar_data.head(10)
# elment in 1:end columns is the range (distance in meter)

'\nFOV: 190 (degree)\nStart angle: -5 (degree)\nEnd angle: 185 (degree)\nAngular resolution: 0.666 (degree) # shoot a laser ray every 0.666 degree\nMax range: 80 (meter)\n\n* LiDAR rays with value 0.0 represent infinite range observations.\n'

In [4]:
# Only consider points between the minimum and maximum range of the sensor.
# Measurements between 2m-75m are recommended to be included as valid data.


In [5]:
'''
lidar_data = lidar_data.drop(["timestamp"], axis = 1)
lidar_data.head()
lidar_data.plot.hist(bins=10)
plt.show()
'''

'\nlidar_data = lidar_data.drop(["timestamp"], axis = 1)\nlidar_data.head()\nlidar_data.plot.hist(bins=10)\nplt.show()\n'

In [6]:
##################################################################################################################

## Encoder

In [7]:
'''
Encoder calibrated parameter
Encoder resolution: 4096
Encoder left wheel diameter: 0.623479
Encoder right wheel diameter: 0.622806
Encoder wheel base: 1.52439

* The encoder data is stored as [timestamp, left count, right count].
'''
print(encoder_data.shape)
encoder_data.head(10)

(116048, 3)


,timestamp,left_count,right_count
0,1544582648751852447,69577,69699
1,1544582648761750060,69673,69795
2,1544582648771748778,69769,69891
3,1544582648781765037,69865,69986
4,1544582648791768265,69961,70081
5,1544582648801801135,70057,70178
6,1544582648811766755,70154,70273
7,1544582648821765891,70251,70369
8,1544582648831862776,70348,70467
9,1544582648841781008,70445,70565


In [8]:
df = encoder_data.copy()
print(df.shape)
df.head()

(116048, 3)


,timestamp,left_count,right_count
0,1544582648751852447,69577,69699
1,1544582648761750060,69673,69795
2,1544582648771748778,69769,69891
3,1544582648781765037,69865,69986
4,1544582648791768265,69961,70081


In [9]:
df = np.diff(df, axis=0)
df

array([[ 9897613,       96,       96],
       [ 9998718,       96,       96],
       [10016259,       96,       95],
       ...,
       [10004866,      142,      143],
       [ 9997784,      142,      143],
       [10000575,      143,      142]])

In [10]:
new = {'dt': df[:,0]*1e-9, 'dlz': df[:,1], 'drz': df[:,2]}
new_df = pd.DataFrame(data=new)
new_df.head()

,dt,dlz,drz
0,0.009898,96,96
1,0.009999,96,96
2,0.010016,96,95
3,0.010003,96,95
4,0.010033,96,97


In [11]:
res = 4096
dL = 0.623479
dR = 0.622806

In [12]:
new_df["distance_left"] = np.pi * dL/res * new_df["dlz"]
new_df["distance_right"] = np.pi * dR/res * new_df["drz"]
new_df['velocity_left'] = new_df["distance_left"] /  new_df["dt"]
new_df['velocity_right'] = new_df["distance_right"] /  new_df["dt"]
new_df['linear_velocity(m/s)'] = (new_df['velocity_left'] + new_df['velocity_right'])/2.0 
# new_df['linear_velocity(mph)'] = new_df['linear_velocity(m/s)']*2.237
new_df.head()

,dt,dlz,drz,distance_left,distance_right,velocity_left,velocity_right,linear_velocity(m/s)
0,0.009898,96,96,0.045907,0.045858,4.638233,4.633226,4.635729
1,0.009999,96,96,0.045907,0.045858,4.591332,4.586376,4.588854
2,0.010016,96,95,0.045907,0.045380,4.583291,4.530653,4.556972
3,0.010003,96,95,0.045907,0.045380,4.589262,4.536555,4.562908
4,0.010033,96,97,0.045907,0.046336,4.575703,4.618376,4.597039


In [13]:
new_df.describe()

,dt,dlz,drz,distance_left,distance_right,velocity_left,velocity_right,linear_velocity(m/s)
count,116047.000000,116047.000000,116047.000000,116047.000000,116047.000000,116047.000000,116047.000000,116047.000000
mean,0.010000,94.920937,94.844046,0.045391,0.045306,4.539410,4.530837,4.535123
std,0.000102,102.020106,102.089331,0.048786,0.048767,4.879223,4.877265,4.878008
min,0.009454,-2.000000,-2.000000,-0.000956,-0.000955,-0.100529,-0.096457,-0.095616
25%,0.009988,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.010000,56.000000,56.000000,0.026779,0.026750,2.676730,2.673866,2.674156
75%,0.010012,184.000000,184.000000,0.087989,0.087894,8.812324,8.811276,8.814739
max,0.010616,309.000000,557.000000,0.147765,0.266071,15.175901,26.624950,20.514244


In [14]:
sum(new_df['distance_left']), sum(new_df['distance_right'])

(5267.538156830349, 5257.589840347711)

In [15]:
##################################################################################################################

## FOG 

In [16]:
'''
FOG (Fiber Optic Gyro) extrinsic calibration parameter from vehicle
RPY(roll/pitch/yaw = XYZ extrinsic, degree), R(rotation matrix), T(translation matrix, meter)
RPY: 0 0 0
R: 1 0 0 0 1 0 0 0 1 
T: -0.335 -0.035 0.78

* The sensor measurements are stored as [timestamp, delta roll, delta pitch, delta yaw] in radians.
'''
print(fog_data.shape)
fog_data.head(10)

(1160508, 4)


,timestamp,delta_roll,delta_pitch,delta_yaw
0,1544582648743131772,-0.000005,0.000028,-0.000012
1,1544582648744114218,-0.000007,0.000025,-0.000011
2,1544582648745131427,0.000002,0.000022,-0.000010
3,1544582648746103145,0.000017,0.000022,-0.000010
4,1544582648747106429,0.000031,0.000023,-0.000010
5,1544582648748126535,0.000039,0.000022,-0.000010
6,1544582648749104662,0.000035,0.000021,-0.000010
7,1544582648750131486,0.000021,0.000017,-0.000010
8,1544582648751099508,0.000003,0.000013,-0.000009
9,1544582648752131818,-0.000015,0.000006,-0.000009
